In [3]:
from terminusdb_client import WOQLClient, WOQLQuery

db_id = "nlp_spacy"
client = WOQLClient(server_url = "http://localhost:6363")
client.connect(key="root", account="admin", user="admin")
client.db(db_id)

'nlp_spacy'

In [5]:
import spacy
#import pprint as pp

def doc_id(doc, var_name):
    return WOQLQuery().idgen("doc:SpaCyDoc",
                              [WOQLQuery().string(doc.text)], var_name)

def token_id(doc, token, var_name):
    return WOQLQuery().idgen("doc:SpaCyToken",
                              [WOQLQuery().string(doc.text),
                              WOQLQuery().string(str(token.i))], var_name)
    

nlp = spacy.load("en_core_web_sm")
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")

insert_doc = doc_id(doc, "v:doc_id") + WOQLQuery().insert("v:doc_id", "scm:SpaCyDoc", label=doc.text)

insert_doc.execute(client, "Add in the doc")

all_token = []
for token in doc:
    print(token.text, token.lemma_, token.pos_, token.tag_, token.dep_,
            token.shape_, token.is_alpha, token.is_stop)
    insert_token = (WOQLQuery().insert(f"v:token_id_self{token.i}", "scm:SpaCyToken", label=token.text)
                               .property("lemma", token.lemma_)
                               .property("pos", token.pos_)
                               .property("tag", token.tag_)
                               .property("dep", token.dep_)
                               .property("shape", token.shape_)
                               .property("is_alpha", token.is_alpha)
                               .property("is_stop", token.is_stop)
                               .property("head", f"v:token_id_head{token.i}")
                               .property("doc", f"v:token_doc_id{token.i}")
                   )
    all_token.append(
                     token_id(doc, token, f"v:token_id_self{token.i}")
                     + token_id(doc, token.head, f"v:token_id_head{token.i}")
                     + doc_id(token.doc, f"v:token_doc_id{token.i}")
                     + insert_token
                     )
WOQLQuery().woql_and(*all_token).execute(client, "Add in all the tokens")

Autonomous autonomous ADJ JJ amod Xxxxx True False
cars car NOUN NNS nsubj xxxx True False
shift shift VERB VBP ROOT xxxx True False
insurance insurance NOUN NN compound xxxx True False
liability liability NOUN NN dobj xxxx True False
toward toward ADP IN prep xxxx True True
manufacturers manufacturer NOUN NNS pobj xxxx True False


{'bindings': [{'token_doc_id0': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id1': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id2': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id3': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id4': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id5': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_doc_id6': 'terminus:///terminus/document/SpaCyDoc_Autonomous%20cars%20shift%20insurance%20liability%20toward%20manufacturers',
   'token_id_head0': 'terminu

In [6]:
for token in doc:
    print(token.text, token.head.text)

Autonomous cars
cars shift
shift shift
insurance liability
liability shift
toward liability
manufacturers toward


In [ ]:
# Delete everything
# WOQLQuery().triple("v:A","v:B","v:C").delete_triple("v:A","v:B","v:C").execute(client, "Delete everything")

In [ ]:
import spacy

nlp = spacy.load("en_core_web_sm")
doc = nlp("Autonomous cars shift insurance liability toward manufacturers")
for token in doc:
    print(token.text, token.dep_, token.head.text, token.head.pos_,
            [child for child in token.children])